In [1]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

In [2]:
import tensorflow as tf

In [3]:
tf.random.set_seed(1234)

In [4]:

modell = Sequential()
modell.add(Bidirectional(LSTM(100, activation='relu', input_shape=(100,1)))) #elu
#modell.add(Bidirectional(LSTM(50, dropout=0.5)))

#modell.add(Bidirectional(LSTM(100, dropout=0.5)))
#modell.add(BatchNormalization(momentum=0.6))
modell.add(Dense(1))
modell.compile(loss='mean_squared_error', optimizer='adam') #rmsprop adam




In [5]:
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()
# For time stamps
from datetime import datetime


# The tech stocks we'll use for this analysis
tech_list = ['0P00005V63.BO','0P0000MLHH.BO','0P0000XVYQ.BO','0P0000P0VW.BO','0P00005VAE.BO','0P0000K8MR.BO','0P0000XW57.BO','0P00005WNE.BO','0P0000GB48.BO','0P0000VZ4S.BO','0P0000XV0H.BO','0P0000KUO1.BO','0P0001L80T.BO','0P00005WRH.BO','0P00005WVM.BO','0P0000AEJZ.BO','0P00009JAQ.BO','0P0001B9WM.BO','0P0000XVJQ.BO','0P0001KN75.BO','0P00005UWX.BO','0P00005WIB.BO','0P0001ABMA.BO','0P0001ABMA.BO']

# Set up End and Start times for data grab

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)
i = 1
for stock in tech_list:
    val = 'f' + str(i)
    globals()[val] = yf.download(stock, start='2012-01-01', end=datetime.now())
    i = i + 1

company_list = ['']*(i-1)
k=1
for x in company_list:
    company_list[k-1] = eval('f' + str(k))
    k = k + 1 
#print(company_list)
company_name = ["Aditya Birla Sun Life Frontline Equity Fund","Axis Bluechip Fund","Baroda BNP Paribas Large Cap Fund","Canara Robeco Bluechip Equity Fund","Canara Robeco Bluechip Equity Fund","DSP Top 100 Equity Fund","Edelweiss Large Cap Fund","Franklin India Bluechip Fund","HDFC Top 100 Fund","ICICI Prudential Bluechip Fund","IDBI India Top 100 Equity","IDFC Large Cap Fund","Invesco India Largecap Fund","ITI Large Cap Fund","JM Large Cap Fund","LIC MF Large Cap Fund","Mirae Asset Large Cap Fund","Nippon India Large Cap Fund","PGIM India Large Cap Fund","SBI Bluechip Fund","Sundaram Large Cap Fund","Tata Large Cap Fund","Taurus Largecap Equity Fund","Union Largecap Fund","UTI Mastershare Fund"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df.tail(300)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Open,High,Low,Close,Adj Close,Volume,company_name
Date,,,,,,,
2022-08-04,16.820000,16.820000,16.820000,16.820000,16.820000,0,Union Largecap Fund
2022-08-05,16.870001,16.870001,16.870001,16.870001,16.870001,0,Union Largecap Fund
2022-08-08,16.969999,16.969999,16.969999,16.969999,16.969999,0,Union Largecap Fund
2022-08-10,16.990000,16.990000,16.990000,16.990000,16.990000,0,Union Largecap Fund
2022-08-11,17.110001,17.110001,17.110001,17.110001,17.110001,0,Union Largecap Fund
...,...,...,...,...,...,...,...
2023-10-18,19.420000,19.420000,19.420000,19.420000,19.420000,0,Union Largecap Fund
2023-10-19,19.389999,19.389999,19.389999,19.389999,19.389999,0,Union Largecap Fund
2023-10-20,19.320000,19.320000,19.320000,19.320000,19.320000,0,Union Largecap Fund


In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Scale the data
from sklearn.preprocessing import MinMaxScaler

for i, company in enumerate(company_list, 1):
    # Create a new dataframe with only the 'Close column 
    data = company.filter(['Close'])
    # Convert the dataframe to a numpy array
    dataset = data.values
    # Get the number of rows to train the model on
    training_data_len = int(np.ceil( len(dataset) * .95 ))
    print(str(i)+'. ')
    print(company_name[i-1])
    print(training_data_len)
    print("\n")
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    # scaled_data
    # Create the training data set 
    # Create the scaled training data set
    train_data = scaled_data[0:int(training_data_len), :]
    # Split the data into x_train and y_train data sets
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
        if i<= 61:
            ko=1
            #print(x_train)
            #print(y_train)
            #print()

    # Convert the x_train and y_train to numpy arrays 
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    # x_train.shape

    # Build the LSTM model
#     model = Sequential()
#     model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
#     model.add(LSTM(64, return_sequences=False))
#     model.add(Dense(25))
#     model.add(Dense(1))
   
    # Create the testing data set
    # Create a new array containing scaled values from index 1543 to 2002 
    test_data = scaled_data[training_data_len - 60: , :]
    # Create the data sets x_test and y_test
    x_test = []
    y_test = dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=30, 
            verbose=1, mode='auto', restore_best_weights=True)

    history1=model.fit(x_train,y_train,validation_data=(x_test,y_test),
            callbacks=[monitor],verbose=1,epochs=10)
    
    # Get the models predicted price values 
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    mape = (np.mean(abs((y_test-predictions)/y_test)))*100
    print("Mean Absolute Percentage Error: ")
    print(mape)
    print("\n")
    mae = (np.mean(abs(y_test-predictions)))
    print("Mean Absolute Error: ")
    print(mae)
    print("\n")
    # Get the root mean squared error (RMSE)
    rmse = np.sqrt(np.mean((((y_test - predictions)/y_test) ** 2)))
    print("Root mean square error: ")
    print(rmse)
    print("\n")
    mse = (np.mean((y_test-predictions)**2))
    print("Mean Squared Error: ")
    print(mse)
    print("\n")

1. 
Aditya Birla Sun Life Frontline Equity Fund
1360


Epoch 1/10
41/41 [==============================] - 2s 57ms/step - loss: 2.9069e-04 - val_loss: 151560.6406
Epoch 2/10
41/41 [==============================] - 2s 56ms/step - loss: 2.8842e-04 - val_loss: 151560.1094
Epoch 3/10
41/41 [==============================] - 2s 55ms/step - loss: 2.7567e-04 - val_loss: 151562.0000
Epoch 4/10
41/41 [==============================] - 2s 56ms/step - loss: 2.4189e-04 - val_loss: 151558.1406
Epoch 5/10
41/41 [==============================] - 2s 56ms/step - loss: 2.4970e-04 - val_loss: 151560.8594
Epoch 6/10
41/41 [==============================] - 2s 56ms/step - loss: 2.3521e-04 - val_loss: 151563.8906
Epoch 7/10
41/41 [==============================] - 2s 56ms/step - loss: 2.2546e-04 - val_loss: 151554.6406
Epoch 8/10
41/41 [==============================] - 2s 57ms/step - loss: 2.4952e-04 - val_loss: 151549.8750
Epoch 9/10
41/41 [==============================] - 2s 58ms/step - loss: 2.2383e-

KeyboardInterrupt: 